In [36]:
# Libraries
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
from hda import * # this is the PYTHON version

import numpy as np
import pandas as pd



In [37]:
depth = pd.read_csv("sea_depth2.csv", delimiter=',', decimal='.')

In [38]:
depth

,y,x,latitude,longitude,model_depth
0,-27.375,22.250,59.083620,-5.896293e+00,72.649900
1,-27.375,22.375,59.017727,-5.739098e+00,74.176270
2,-27.375,22.500,58.951637,-5.582606e+00,86.539060
3,-27.375,22.625,58.885345,-5.426813e+00,87.302246
4,-27.375,22.750,58.818860,-5.271716e+00,83.639160
...,...,...,...,...,...
465,-24.625,24.500,59.534790,-1.457905e-01,119.811520
466,-24.625,24.625,59.460980,3.382219e-09,115.996090
467,-24.625,24.750,59.387005,1.450523e-01,107.906740
468,-24.625,24.875,59.312874,2.893702e-01,125.306150


In [39]:
sea_v = pd.read_csv("sea_v2.csv", delimiter=',', decimal='.')

In [40]:
sea_v

,time,depth,y,x,latitude,longitude,speed
0,2019-01-01,0.0,-27.375,22.250,59.083620,-5.896293e+00,0.150979
1,2019-01-01,0.0,-27.375,22.375,59.017727,-5.739098e+00,0.168134
2,2019-01-01,0.0,-27.375,22.500,58.951637,-5.582606e+00,0.096445
3,2019-01-01,0.0,-27.375,22.625,58.885345,-5.426813e+00,0.049326
4,2019-01-01,0.0,-27.375,22.750,58.818860,-5.271716e+00,0.044866
...,...,...,...,...,...,...,...
14565,2019-01-31,0.0,-24.625,24.500,59.534790,-1.457905e-01,0.079068
14566,2019-01-31,0.0,-24.625,24.625,59.460980,3.382219e-09,0.089209
14567,2019-01-31,0.0,-24.625,24.750,59.387005,1.450523e-01,0.098422
14568,2019-01-31,0.0,-24.625,24.875,59.312874,2.893702e-01,0.095668


In [69]:
sea_depth = []
counter = 0
for row in range(len(sea_v)): 
    value = depth[(depth["latitude"] == sea_v["latitude"][row]) & (depth["longitude"] == sea_v["longitude"][row])]["model_depth"].tolist()
    if len(value) !=0:
        sea_depth += value
    else: 
        counter += 1
counter

0

In [84]:
sea_v["sea_depth"] = sea_depth 
sea_v

,time,depth,y,x,latitude,longitude,speed,sea_depth
0,2019-01-01,0.0,-27.375,22.250,59.083620,-5.896293e+00,0.150979,72.649900
1,2019-01-01,0.0,-27.375,22.375,59.017727,-5.739098e+00,0.168134,74.176270
2,2019-01-01,0.0,-27.375,22.500,58.951637,-5.582606e+00,0.096445,86.539060
3,2019-01-01,0.0,-27.375,22.625,58.885345,-5.426813e+00,0.049326,87.302246
4,2019-01-01,0.0,-27.375,22.750,58.818860,-5.271716e+00,0.044866,83.639160
...,...,...,...,...,...,...,...,...
14565,2019-01-31,0.0,-24.625,24.500,59.534790,-1.457905e-01,0.079068,119.811520
14566,2019-01-31,0.0,-24.625,24.625,59.460980,3.382219e-09,0.089209,115.996090
14567,2019-01-31,0.0,-24.625,24.750,59.387005,1.450523e-01,0.098422,107.906740
14568,2019-01-31,0.0,-24.625,24.875,59.312874,2.893702e-01,0.095668,125.306150


In [72]:
sea_v.to_csv("Speed_depth.csv")

In [85]:
sea_v_filter = sea_v[(sea_v["sea_depth"]>40) & (sea_v["sea_depth"]<200)]

In [87]:
sea_v_filter.to_csv("Speed_depth_filter.csv")

In [110]:
sea_v_filter = sea_v_filter.reset_index()

In [80]:
# Power calculation 
def tidalPower(n = 1, dens = 1025, area = 10, v = 1, maxV = 5):
    """
    P = Power generated in W
    n = turbine efficiency [0-1]
    dens = water density in kg/m^3
    area = sweep area of the turbine in m^2
    v = velocity of the flow in m/s
    """
    if v < maxV:
        P = n*dens*area*v**3/2
    else:
        P = 0
    return P

def wavesPower(H = 5, Te = 10, dens = 1025, g = 9.8, maxH = 8):
    """
    P  = wave energy flux per unit of wave-crest length
    H = the significant wave height in m
    Te = the wave energy period in s
    dens = the water density
    g  = the acceleration by gravity
    
    When the significant wave height is given in meters, 
    and the wave period in seconds, the result is the wave power in kilowatts (kW) 
    per meter of wave front length.
    """
    if H < maxH:
        P = dens*g**2*H**2*Te/64*3.14
    else:
        P = 0
    return P
    
def energyProduced(passedTime = None, power = None):
    e = power/3600*passedTime
    return e

In [125]:
# Possible power calculation 
nTurbine = 0.4
areaTurbine = 5**2 * 3.14
# velocity = 1.5
out_energy = []

for row in range(len(sea_v_filter)):
    velocity = sea_v_filter["speed"][row]
    totalEnergy = 0
    for i in range(24): #in 24 hours
        P = tidalPower(n = nTurbine, area = areaTurbine, v = velocity)
        energy = energyProduced(3600, P)
        totalEnergy = (totalEnergy + energy)
        #print(P/1000)
    out_energy += [totalEnergy]

# print(totalEnergy/1000,"kWh")
# out_energy

In [126]:
sea_v_filter["energy"] = out_energy

In [116]:
sea_v_filter.to_csv("Energy_per_day.csv")

In [120]:
days = sea_v_filter["time"].unique()

In [127]:
sea_v_filter[sea_v_filter["time"] == days[0]]

,index,time,depth,y,x,latitude,longitude,speed,sea_depth,energy
0,0,2019-01-01,0.0,-27.375,22.250,59.083620,-5.896293e+00,0.150979,72.649900,1329.184199
1,1,2019-01-01,0.0,-27.375,22.375,59.017727,-5.739098e+00,0.168134,74.176270,1835.703591
2,2,2019-01-01,0.0,-27.375,22.500,58.951637,-5.582606e+00,0.096445,86.539060,346.479404
3,3,2019-01-01,0.0,-27.375,22.625,58.885345,-5.426813e+00,0.049326,87.302246,46.352048
4,4,2019-01-01,0.0,-27.375,22.750,58.818860,-5.271716e+00,0.044866,83.639160,34.880847
...,...,...,...,...,...,...,...,...,...,...
445,465,2019-01-01,0.0,-24.625,24.500,59.534790,-1.457905e-01,0.196768,119.811520,2942.390832
446,466,2019-01-01,0.0,-24.625,24.625,59.460980,3.382219e-09,0.216902,115.996090,3941.190356
447,467,2019-01-01,0.0,-24.625,24.750,59.387005,1.450523e-01,0.234099,107.906740,4954.873196
448,468,2019-01-01,0.0,-24.625,24.875,59.312874,2.893702e-01,0.240310,125.306150,5359.831085


In [124]:
max(sea_v_filter["energy"])

2.253219943258279